## Practical Assignment 5 - POS(Part of Speech) Tagging

In [4]:
import nltk

In [ ]:
# most basic model is the bag of words model
I LIKE YOU
I AM LIKE YOU #LIKE functioning differently in both cases
#so by using pos we are trying yo understand how a word is being used in a sentence
#POS tagging also used for building lemmatisers.

In [ ]:
#we are doing POS tagging using the CRF CLASSIFIER

In [5]:
conda install -c conda-forge sklearn-crfsuite

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\ritvi\anaconda3

  added / updated specs:
    - sklearn-crfsuite


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.0               |   py37h03978a9_1         3.1 MB  conda-forge
    python-crfsuite-0.9.7      |   py37h1fb7aa8_1         175 KB  conda-forge
    sklearn-crfsuite-0.3.6     |     pyh9f0ad1d_0          12 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be INSTALLED:

  python-crfsuite    conda-forge/win-64::python-crfsuite-0.9.7-py37h1fb7aa8_1
  sklearn-crfsuite   conda-forge/noarch::sklearn-crfsuite-0.3.6-pyh9f0ad1d_0

The following packages will be UPDATED:

  conda                                4.9.2-py37h03978a9_0 --> 4.10.0-py37h03978a9_1


In [6]:
import nltk
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF,metrics
from collections import Counter
print('POS Tagging using CRF')

POS Tagging using CRF


In [7]:
tagged_sentences = nltk.corpus.treebank.tagged_sents(tagset='universal')#getting a pre-existing database from the nltk corpus

In [8]:
tagged_sentences #so we have a database where every word is already tagged

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], ...]

In [9]:
len(tagged_sentences) #no of sentences

3914

In [10]:
print("Tagged words : ",len(nltk.corpus.treebank.tagged_words()))#no of words

Tagged words :  100676


In [12]:
tagged_words = [tup for sent in tagged_sentences for tup in sent]
print("Tagged words : ",len(nltk.corpus.treebank.tagged_words()))

Tagged words :  100676


In [13]:
tags = set([tag for word,tag in tagged_words])

In [14]:
tags #to get all the universal tags in the dataset

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [15]:
train_set,test_set = train_test_split(tagged_sentences,test_size=0.2)

In [16]:
len(train_set)

3131

In [17]:
len(test_set)

783

In [18]:
train_set[0]

[('``', '.'),
 ('If', 'ADP'),
 ('we', 'PRON'),
 ('have', 'VERB'),
 ('a', 'DET'),
 ('real', 'ADJ'),
 ('bad', 'ADJ'),
 ('day', 'NOUN'),
 (',', '.'),
 ('the', 'DET'),
 ('program', 'NOUN'),
 ('would', 'VERB'),
 ('say', 'VERB'),
 (',', '.'),
 ('`', '.'),
 ('*', 'X'),
 ('Buy', 'VERB'),
 (',', '.'),
 ("'", '.'),
 ("''", '.'),
 ('he', 'PRON'),
 ('explains', 'VERB'),
 ('*T*-1', 'X'),
 ('.', '.')]

In [20]:
#[{"metadata":{"trusted":true},"cell_type":"code","source":"tagged_words=[tup for sent in tagged_sentences for tup in sent]\nprint(len(tagged_words))\nprint(\"Tagged words:\", len(nltk.corpus.treebank.tagged_words()))","execution_count":12,"outputs":[{"output_type":"stream","text":"100676\nTagged words: 100676\n","name":"stdout"}]}]

In [21]:
#[{"metadata":{"trusted":true},"cell_type":"code","source":"tagged_words=[tup for sent in tagged_sentences for tup in sent]\nprint(len(tagged_words))\n
#print(\"Tagged words:\", len(nltk.corpus.treebank.tagged_words()))","execution_count":12,"outputs":[{"output_type":"stream","text":"100676\nTagged words: 100676\n","name":"stdout"}]}]

In [19]:
#define the fetures
#we are passing the sentence and the words in that particular sentence in the feature function. if there are 4 words then the index will be 0 to 3
def features(sentence,index):
    return{
        'is_first_capital':int(sentence[index][0].isupper()),
        'is_first_word':int(index==0),
        'is_last_word':int(index==len(sentence)-1),
        'is complete_capital':int(sentence[index].upper()==sentence[index]),
        'prev_word':'' if index==0 else sentence[index-1],
        'next_word':'' if index==len(sentence)-1 else sentence[index + 1],
        'is_numeric': sentence[index].isdigit(),
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'prefix-4': sentence[index][:4],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'suffix-4': sentence[index][-3:],
        'has_hyphen': '-' in sentence[index]
    }  
#note that for every word these many features are going to get generated

In [20]:
def untag(sentence): #we have to untag the sentence because we dont want tagged sentences for testing purposes
    return[word for word,tag in sentence]
def prepareData(tagged_Sentences): #preparing data for training set and test set
    x,y=[],[]
    for sentences in tagged_sentences:
        x.append([features(untag(sentences),index) for index in range(len(sentences))])
        y.append([tag for word,tag in sentences])
    return x,y
#y will have the word and y will have the tag. doing it for both training and testing dataset


In [21]:
x_train,y_train = prepareData(train_set)
x_test,y_test = prepareData(test_set)

In [22]:
print('X Training Data',x_train[0]) #for each and every word we are generating the features to classify.

X Training Data [{'is_first_capital': 1, 'is_first_word': 1, 'is_last_word': 0, 'is complete_capital': 0, 'prev_word': '', 'next_word': 'Vinken', 'is_numeric': False, 'prefix-1': 'P', 'prefix-2': 'Pi', 'prefix-3': 'Pie', 'prefix-4': 'Pier', 'suffix-1': 'e', 'suffix-2': 're', 'suffix-3': 'rre', 'suffix-4': 'rre', 'has_hyphen': False}, {'is_first_capital': 1, 'is_first_word': 0, 'is_last_word': 0, 'is complete_capital': 0, 'prev_word': 'Pierre', 'next_word': ',', 'is_numeric': False, 'prefix-1': 'V', 'prefix-2': 'Vi', 'prefix-3': 'Vin', 'prefix-4': 'Vink', 'suffix-1': 'n', 'suffix-2': 'en', 'suffix-3': 'ken', 'suffix-4': 'ken', 'has_hyphen': False}, {'is_first_capital': 0, 'is_first_word': 0, 'is_last_word': 0, 'is complete_capital': 1, 'prev_word': 'Vinken', 'next_word': '61', 'is_numeric': False, 'prefix-1': ',', 'prefix-2': ',', 'prefix-3': ',', 'prefix-4': ',', 'suffix-1': ',', 'suffix-2': ',', 'suffix-3': ',', 'suffix-4': ',', 'has_hyphen': False}, {'is_first_capital': 0, 'is_first_

In [23]:
print('Y Training Data',y_train[0])

Y Training Data ['NOUN', 'NOUN', '.', 'NUM', 'NOUN', 'ADJ', '.', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', 'NUM', '.']


In [24]:
#crf is method based on gradient descent using l1 and l2 regularisation. you can check google
crf=CRF(
    algorithm='lbfgs',
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(x_train,y_train)

C:\Users\ritvi\anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.01, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [ ]:
#so the modelling part is now over.

In [26]:
crf.classes_

['NOUN',
 '.',
 'NUM',
 'ADJ',
 'VERB',
 'DET',
 'ADP',
 'CONJ',
 'X',
 'ADV',
 'PRT',
 'PRON']

In [27]:
y_pred = crf.predict(x_test)

In [28]:
print(metrics.flat_classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           .       1.00      1.00      1.00     11715
         ADJ       0.98      0.98      0.98      6397
         ADP       0.99      1.00      0.99      9857
         ADV       0.98      0.97      0.98      3171
        CONJ       1.00      1.00      1.00      2265
         DET       1.00      1.00      1.00      8725
        NOUN       0.99      1.00      0.99     28867
         NUM       1.00      1.00      1.00      3546
        PRON       1.00      1.00      1.00      2737
         PRT       0.99      0.99      0.99      3219
        VERB       1.00      0.99      0.99     13564
           X       1.00      1.00      1.00      6613

    accuracy                           0.99    100676
   macro avg       0.99      0.99      0.99    100676
weighted avg       0.99      0.99      0.99    100676



In [29]:
metrics.flat_accuracy_score(y_test,y_pred)

0.9948845802375939

In [30]:
Counter(crf.state_features_).most_common(10) #to see the top 10 features

[(('prefix-4:many', 'ADJ'), 7.773886),
 (('prev_word:will', 'VERB'), 7.499126),
 (('prev_word:would', 'VERB'), 6.490764),
 (('prefix-1:*', 'X'), 6.246508),
 (('suffix-2:ly', 'ADV'), 6.014297),
 (('prev_word:could', 'VERB'), 5.536372),
 (('prefix-4:Many', 'ADJ'), 5.534789),
 (('is_first_capital', 'NOUN'), 5.510457),
 (('prev_word:to', 'VERB'), 4.999606),
 (('next_word:Express', 'NOUN'), 4.658671)]